In [57]:
# 1. 데이터 준비

import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

  # txt 파일을 모두 읽어서 raw_corpus에 담기
for txt_file in txt_list:
  with open(txt_file, "r") as f:
    raw = f.read().splitlines()

    # list.extend(iterable) : list 끝에 iterable 객체 삽입 (참고 append는 리스트 자체를 삽입)
    # append와 extend 차이 : https://m.blog.naver.com/wideeyed/221541104629
    raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])


데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [87]:
# 2. 데이터 정제

import re
import numpy as np
import tensorflow as tf


  # 토큰화를 위한 데이터 정리 함수 생성 
    # 대문자를 소문자로 변환 : lower() 
    # 양쪽 끝 공백 제거 : strip()
    # re.sub(r"A","B") : 앞에 r이 붙을 경우 해당 문자열을 그대로 출력 
def preprocess_sentence(sentence):
  # 대문자를 소문자로 바꾸고, 양쪽 끝 공백제거
  sentence = sentence.lower().strip()
  # 특수문자 양쪽에 공백 넣기
  sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
  # 여러개의 공백은 하나의 공백으로 바꾸기
  sentence = re.sub(r'[" "]', " ", sentence)
  # a-zA-Z?.!,¿가 아닌 모든 문자는 하나의 공백으로 바꿈
  sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
  # 다시 양쪽 공백을 지움
  sentence = sentence.strip()
  # 문장 시작에는 <start>, 끝에는 <end>를 추가
  sentence = '<start> ' + sentence + ' <end>'
  # 단어 개수가 15개 이상인 문장은 공백으로 바꾸기
  if len(sentence.split()) > 15 : 
    sentence = ""
  

  return sentence



  # 데이터 정제
    # 1. 길이가 0인 문장과, 15인 문장 화자를 나타내는 문장 건너 뛰기
    # 2. 함수를 이용하여 데이터 정제 후 리스트에 담기

corpus = []
for sentence in raw_corpus:
  if len(sentence)==0 : continue
  if sentence[-1] == ":" : continue


  preprossed_sentence = preprocess_sentence(sentence)
  if len(preprocessed_sentence) == 0: continue 

  corpus.append(preprossed_sentence)

  # 정제된 데이터 샘플(10개) 확인
corpus[:10]
print(len(corpus))



  # 토큰화 및 텐서 변환
    # 1. tokenize 함수 생성 
      # 1) tokenizer 선언 : tf.keras.preprocessing.text.Tokenizer(num_words= , filters= , ovv_token="<unk>")
      # 2) tokenize 후 단어장 생성 : tokenizer.fit_on_texts()
      # 3) Tensor로 변환 : tokenizer.texts_to_sequences()
      # 4) 입력 데이터의 시퀀스 길이 일정하게 맞추기 : tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
      # 5) tensor 및 tokenizer 프린트 및 리턴
    # 2. tokenize 실행

def tokenize(corpus):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(
      num_words = 12000,
      filters='',
      oov_token = "<unk>")
  tokenizer.fit_on_texts(corpus)
  tensor = tokenizer.texts_to_sequences(corpus)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post')
  print(tensor, tokenizer)
  return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

  # (데이터 확인) 텐서 데이터 3번째 행, 10번째 열까지 출력
print("-----------------------------------------")
print(tensor[:3, :10])



  # (데이터 확인) tokenizer 단어사전 내 index 10번까지 출력
print("-----------------------------------------")
for idx in tokenizer.index_word:
  print(idx, ":", tokenizer.index_word[idx])
  if idx >= 10: break 


175749
[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f4a9ba37730>
-----------------------------------------
[[   2   50    4   95  303   62   53    9  946 6263]
 [   2   15 2967  871    5    8   11 5739    6  374]
 [   2   33    7   40   16  164  288   28  333    5]]
-----------------------------------------
1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [88]:
# 3. 평가 데이터셋 분리

  # 소스 문장 및 타깃 문장 정의
    # 소스 문장 : <end> 제거
    # 타깃 문장 : <start> 제거
    # 데이터 확인
scr_input = tensor[ : , : -1]
tgt_input = tensor[ : , 1 : ]
print(scr_input[10])
print(tgt_input[10])



  # 훈련 데이터와 평가 데이터 분리
    # 모듈 임포트
    # train_test_split 
    # validation data 20%로 분리
    # train 데이터 shape 출력
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(scr_input, tgt_input, test_size=0.2, random_state=21)
print("Source Train: ", enc_train.shape)
print("Target Train: ", dec_train.shape)



  # 데이터셋 만들기
    # 1. BUFFER_SIZE, BATCH_SIZE, step_per_epoch(=len(scr_input)//BATCH_SIZE) 설정
    # 2. VOCAB_SIZE 설정 (tonkenizer.num_words)
    # 3. 데이터셋 만들기 
      # 데이터셋 생성 : tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
      # 데이터셋 섞기? : .shuffle(BUFFER_SIZE)
      # 데이터셋 배치사이즈 결정 : .batch(BATCH_SIZE, drop_remainder = True)
BUFFER_SIZE = len(scr_input)
BATCH_SIZE = 256
step_per_epoch = len(scr_input) / BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)

[   2    7  467   65 7665   18    6 1564    3    0    0    0    0    0]
[   7  467   65 7665   18    6 1564    3    0    0    0    0    0    0]
Source Train:  (140599, 14)
Target Train:  (140599, 14)


In [89]:
# 4. 인공지능 만들기

  # 모델 생성 (Class TextGenerator(tf.keras.Model)
    # 자식클래스 함수 생성(부모 클래스 tf.keras.Model)
      # 임베딩 : tf.keras.layers.Embedding(vocab_size, embedding_size)
      # rnn_1, 2 : tf.keras.layers.LSTM(hidden_size, return_sequences=True)
      # linear : tf.keras.layers.Dense(vocab_size)
    # call 함수 생성
class TextGenerator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_size, hidden_size):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
    self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
    self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
    self.linear = tf.keras.layers.Dense(vocab_size)

  def call(self,x):
    out = self.embedding(x)
    out = self.rnn_1(out)
    out = self.rnn_2(out)
    out = self.linear(out)

    return out

  # embedding_size, hidden_size 값 설정
embedding_size = 256
hidden_size = 1024



  # 클래스 호출 및 모델 정의
model = TextGenerator(tokenizer.num_words +1, embedding_size, hidden_size)  



  # (테스트) 모델 테스트
    # 배치 하나 불러오기 
    # 모델에 넣기
    # 모델 서머리 확인
for scr_sample, tgt_sample in dataset.take(1) : break
model(scr_sample)
model.summary()



  # 모델 학습
    # optimizer 생성 : tf.keras.optimizers.Adam()
    # loss 생성 : tf.keras.SparseCategoricalCrossentropy(from_logits=True, reduction = 'none')
    # 모델 컴파일 : model.compile(loss=, optimizer=)
    # 모델 학습 : model.fit(dataset , epochs=)
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none')
model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)




Model: "text_generator_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_14 (LSTM)               multiple                  5246976   
_________________________________________________________________
lstm_15 (LSTM)               multiple                  8392704   
_________________________________________________________________
dense_7 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
549/549 [==============================] - 103s 184ms/step - loss: 3.1015
Epoch 2/10
549/549 [==============================] - 100s 182ms/step - loss: 2.6607
Epoch 3/10
549/549 [==============================] - 100s 182ms/s

In [100]:
# 작문 모델 생성
  
  # 텍스트 생성 함수 정의
    # 함수명 및 변수 지정
    # 테스트 문장 텐서로 변경
      # 테스트 인풋 정의: tokenizer.texts_to_sequences
      # 테스트 텐서 정의 : tf.covert_to_tensor(test_input, dtpye=tf.init64)
      # 마지막 토큰 정의 : end_token = tokenizer.word_index["<end"]
    # while 함수를 이용하여 문장 생성
      # 1. 입력받은 문장의 텐서를 입력
      # 2. 예측된 값 중 가장 높은 확률의 word index 뽑기
      # 3. 예측한 word index를 문장 뒤에 붙임
      # 4. 모델이 <end>를 예측했거나, max_len에 도달하면 문장 생성을 마침
    # tokennizer를 이용해 word index를 단어로 하나씩 변환


def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

  # 가사 생성 
generate_text(model, tokenizer, init_sentence="<start> I like", max_len=20)


'<start> i like the way how you re kissin me <end> '